In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder .getOrCreate()

import pandas as pd
d=pd.read_parquet("data/bizum.parquet")

In [2]:
from qviz.adp import SparkAutomatedPreparation
p = SparkAutomatedPreparation()

In [3]:
p.set_args(remove_outliers=True,
                             threshold_cardinality=15,
                             do_PCA=True)


In [4]:
dropped = spark.createDataFrame(d.drop(
    [a for a in d  if d[a].nunique()==1]
        ,axis=1))



In [5]:
types = {column: t for column, t in dropped.dtypes}


In [6]:
types
df2 = p._encode_categorical(dropped, types)


In [7]:
p.scaling_method=0
df3 = p._scale_df(p._assemble_df(df2))

In [8]:
df4 = p._perform_pca(df3)

In [18]:
df = df3
from pyspark.ml.feature import PCA

n_samples = df.count()
n_features = len(df.select("features").head()[0])
n_components = min(n_samples, n_features)

inputCol = "features"
outputCol = "pca_features"

pca = PCA(k=3, inputCol=inputCol, outputCol=outputCol)
pca_model = pca.fit(df)

accum_variance = 0

for i, var in enumerate(pca_model.explainedVariance):
    accum_variance += var
    if accum_variance >= 0.95:
        components = i + 1
        break
else:
    components = n_components

if components != n_components:
    pca = PCA(k=components, inputCol=inputCol, outputCol=outputCol)
    pca_model = pca.fit(df)

df = pca_model.transform(df)

df = df.drop("features")
df = df.withColumnRenamed("pca_features", "features")


In [28]:
pca_model.pc

DenseMatrix(30, 3, [-0.0305, -0.4021, 0.3313, -0.4052, 0.379, 0.1055, 0.0837, 0.0298, ..., 0.0762, -0.079, 0.0202, 0.2422, 0.143, -0.4461, 0.1286, -0.2488], 0)

In [25]:
f=df.select("features").toPandas()

In [11]:

value=f.apply(lambda a:list(a.features), axis=1, result_type='expand')

In [12]:
f.to_csv("tmp.csv")

In [13]:
value =value.rename(columns={0:'x',1:'y',2:'z'})

In [14]:
final = d.copy()

In [15]:
final['x'] = value.x
final['y'] = value.y
final['z'] = value.z

In [17]:
dropped.columns

['PK_IDE_USUCLO_ORIG',
 'PK_IDENUMSESION',
 'PK_ORDEN',
 'PK_REFERENCIA',
 'IMPORTE_OPERACION',
 'IND_AUTENTICACION_REFORZADA',
 'TIPO_AUTENTICACION',
 'AUTENTICACION_ESPERADA',
 'ESTADO_OP_DEC',
 'ESTADO_OP_DEC_LTX',
 'MOTIVO_ESTADO',
 'ESTADO_OP_TRF',
 'FK_NUMPERSO_TIT_LOE',
 'TERMINAL',
 'HORA_CONFIRMACION']

In [ ]:
from cassandra.cluster import Cluster
c = Cluster()
s = c.connect()
s.execute("DROP KEYSPACE IF EXISTS caixa")
s.execute("DROP KEYSPACE IF EXISTS caixa_qbeast")
s.execute("CREATE KEYSPACE caixa WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1};")
s.execute("""
CREATE TABLE caixa.bizum( 
    PK_ANYOMES          bigint,
    PK_ANYOMESDIA       bigint,
    PK_IDE_USUCLO_ORIG  bigint,
    PK_IDENUMSESION     text, 
    PK_ORDEN            bigint,
    PK_REFERENCIA       text,
    IMPORTE_OPERACION   float,
    IND_AUTENTICACION_REFORZADA      bigint,
    TIPO_AUTENTICACION               text,
    AUTENTICACION_ESPERADA           text,
    ESTADO_OP_DEC                    text,
    ESTADO_OP_DEC_LTX                text,
    MOTIVO_ESTADO                    text,
    ESTADO_OP_TRF                    text,
    NUM_PARTICION                    bigint,
    FK_NUMPERSO_TIT_LOE              bigint,
    COD_TIPO_TERMINAL                text,
    TERMINAL                         bigint,
    EMPRESA                          bigint,
    MOTIVO_ANULACION                 bigint,
    HORA_CONFIRMACION                text,
    FECHA_CARGA                      text,
    x                                float,
    y                                float,
    z                                float,
    PRIMARY KEY(PK_ANYOMES,PK_ANYOMESDIA,PK_IDE_USUCLO_ORIG,PK_IDENUMSESION,PK_REFERENCIA) 
);
""")
#CREATE CUSTOM INDEX  IF NOT EXISTS caixa_bizum_index ON caixa.bizum (x,y,z)
#using 'es.bsc.qbeast.index.QbeastIndex' with OPTIONS = {'transformation': 'LinearTransformation((-120000,140),(-90000,1100),(-50000,50000))'};
s.execute("""CREATE TRIGGER t ON caixa.bizum USING 'es.bsc.qbeast.index.QbeastTrigger';""")
s.execute("""CREATE CUSTOM INDEX  IF NOT EXISTS 
caixa_bizum_index ON caixa.bizum (x,y,z) 
using 'es.bsc.qbeast.index.QbeastIndex' with 
OPTIONS = {'transformation': 'LinearTransformation((-10,10),(-10,10),(-10,10))'};"""
)



In [ ]:
final.to_csv("f_no_index.csv",index=False)

In [ ]:
!cqlsh -e "COPY caixa.bizum(PK_ANYOMES,PK_ANYOMESDIA,PK_IDE_USUCLO_ORIG,PK_IDENUMSESION,PK_ORDEN,PK_REFERENCIA,IMPORTE_OPERACION,IND_AUTENTICACION_REFORZADA,TIPO_AUTENTICACION,AUTENTICACION_ESPERADA,ESTADO_OP_DEC,ESTADO_OP_DEC_LTX,MOTIVO_ESTADO,ESTADO_OP_TRF,NUM_PARTICION,FK_NUMPERSO_TIT_LOE,COD_TIPO_TERMINAL,TERMINAL,EMPRESA,MOTIVO_ANULACION,HORA_CONFIRMACION,FECHA_CARGA,x,y,z) FROM '/home/cesare/workspace/qbeast.io/qviz/examples/f_no_index.csv' with HEADER = true;"
